In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
import matplotlib.pyplot as plt
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [ ]:
count = 0 
dirs = os.listdir('./Dataset')  # Change dataset path here
for dir in dirs:
    path = os.path.join('./Dataset', dir)
    if os.path.isdir(path):  # Ensure it's a directory, not a file (like k.ipynb)
        files = list(os.listdir(path))  # List files inside each category folder
        print(dir + ' Folder has ' + str(len(files)) + ' Images')  
        count += len(files)

print('Images Folder has ' + str(count) + ' Images')  


In [3]:
base_dir = './Dataset'  # Main dataset folder containing Bus, Car, Motorcycle, and Truck
val_dir = './Validation/'  # Adjusted validation dataset folder if you have one
# img_size = 180  # Resize images to 180x180 pixels
img_size = 128
# batch = 32  # Set batch size for training
batch = 16

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory( base_dir,
                                                       seed = 123,
                                                       validation_split=0.2,
                                                       subset = 'training',
                                                       batch_size=batch,
                                                       image_size=(img_size,img_size))

val_ds = tf.keras.utils.image_dataset_from_directory( base_dir,
                                                       seed = 123,
                                                       validation_split=0.2,
                                                       subset = 'validation',
                                                       batch_size=batch,
                                                       image_size=(img_size,img_size))

# Check the number of classes detected
print("Classes:", train_ds.class_names) 

In [ ]:
vehicle_names = train_ds.class_names  # Get the list of class names
print(vehicle_names)  # Print the class names

In [ ]:
i = 0
plt.figure(figsize=(10,10))

for images, labels in train_ds.take(1):
    for i in range(9):
        plt.subplot(3,3, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(vehicle_names[labels[i].numpy()])  
        plt.axis('off')

In [11]:
AUTOTUNE = tf.data.AUTOTUNE

In [12]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [13]:
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal", input_shape = (img_size,img_size,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [ ]:
i = 0
plt.figure(figsize=(10,10))

for images, labels in train_ds.take(1):
    for i in range(9):
        images = data_augmentation(images)
        plt.subplot(3,3, i+1)
        plt.imshow(images[0].numpy().astype('uint8'))
        plt.axis('off')

In [20]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5)
])

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [24]:
model.save('vehicle_classification.keras')


In [ ]:
#  graph and confusion matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Plot Training Accuracy & Loss
plt.figure(figsize=(12, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training & Validation Accuracy')

In [ ]:
# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training & Validation Loss')



In [27]:
plt.show()

In [ ]:
# Confusion Matrix
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=vehicle_names, yticklabels=vehicle_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=vehicle_names))
# ya samma

In [ ]:
def classify_images(image_path):
    input_image = tf.keras.utils.load_img(image_path, target_size=(128,128))
    input_image_array = tf.keras.utils.img_to_array(input_image)
    input_image_exp_dim = tf.expand_dims(input_image_array,0)

    predictions = model.predict(input_image_exp_dim)
    result = tf.nn.softmax(predictions[0])
    outcome = 'The Image belongs to ' + vehicle_names[np.argmax(result)] + ' with a score of '+ str(np.max(result)*100)
    return outcome

tf.keras.models.load_model('vehicle_classification.keras')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imgPath = 'test.jpg'
prediction = classify_images(imgPath)
plt.subplot(3,3,3)
plt.imshow(mpimg.imread(imgPath))
plt.title(prediction)
plt.axis('off')

In [ ]:
# .py file ma matra chalxa.
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image

# Load trained model
model = tf.keras.models.load_model('vehicle_classification.keras')

# Class names
vehicle_names = ['Bus', 'Car', 'Motorcycle', 'Truck']

# Function to classify image
def classify_images(image):
    input_image = image.resize((128, 128))
    input_image_array = np.array(input_image) / 255.0  # Normalize
    input_image_exp_dim = np.expand_dims(input_image_array, axis=0)

    predictions = model.predict(input_image_exp_dim)
    result = tf.nn.softmax(predictions[0])
    
    outcome = f"The image belongs to **{vehicle_names[np.argmax(result)]}** with a confidence score of **{np.max(result) * 100:.2f}%**"
    return outcome

# Streamlit UI
st.title("🚗 Vehicle Classification")
st.write("Upload an image to classify whether it's a Bus, Car, Motorcycle, or Truck.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)
    
    st.write("Classifying...")
    prediction = classify_images(image)
    st.success(prediction)


In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame
import tensorflow as tf
import numpy as np
from PIL import Image, ImageTk

# Load trained model
model = tf.keras.models.load_model('vehicle_classification.keras')

# Class names with emojis
vehicle_names = ['🚌 Bus', '🚗 Car', '🏍️ Motorcycle', '🚛 Truck']

# Function to classify an image
def classify_images(image_path):
    input_image = tf.keras.utils.load_img(image_path, target_size=(128, 128))
    input_image_array = tf.keras.utils.img_to_array(input_image)
    input_image_exp_dim = tf.expand_dims(input_image_array, 0)

    predictions = model.predict(input_image_exp_dim)
    result = tf.nn.softmax(predictions[0])
    
    outcome = f"🎯 This image is a {vehicle_names[np.argmax(result)]} with {np.max(result) * 100:.2f}% accuracy!" 
    return outcome

# Function to open file dialog and classify selected image
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    
    if file_path:
        # Display image
        img = Image.open(file_path)
        img = img.resize((250, 250))  # Resize for display
        img = ImageTk.PhotoImage(img)

        image_label.config(image=img)
        image_label.image = img

        # Classify image
        prediction = classify_images(file_path)
        result_label.config(text=prediction, fg="#FFD700")  # Golden text

# Create Tkinter window
root = tk.Tk()
root.title("🚀 Vehicle Classifier 🏎️")
root.geometry("600x700")
root.configure(bg="#1E3A8A")  # Deep blue background

# Frame for styling
frame = Frame(root, bg="#FFFFFF", bd=5, relief="ridge", highlightbackground="#FFD700", highlightthickness=3)
frame.pack(pady=30, padx=30, fill="both", expand=True)

# Title label
title_label = Label(frame, text="🏎️ Vehicle Classifier 🏁", font=("Arial", 20, "bold"), fg="#1E3A8A", bg="#FFFFFF")
title_label.pack(pady=15)

# Upload button
upload_button = Button(frame, text="📸 Select an Image", command=upload_image, font=("Arial", 14, "bold"), 
                       bg="#FFD700", fg="#1E3A8A", padx=12, pady=8, relief="raised", cursor="hand2")
upload_button.pack(pady=20)

# Image display label
image_label = Label(frame, bg="#FFFFFF", bd=2, relief="ridge")
image_label.pack(pady=15)

# Result label
result_label = Label(frame, text="", font=("Arial", 14, "bold"), fg="#E74C3C", bg="#FFFFFF", wraplength=450, justify="center")
result_label.pack(pady=25)

# Run the GUI
root.mainloop()
